In [1]:
# prompt: adding the google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
def read_data(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.parquet') or f.endswith('.csv')]
    dataframes = {}
    for file in files:
        file_path = os.path.join(folder_path, file)
        if file.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file.endswith('.parquet'):
            df = pd.read_parquet(file_path)
        dataframes[file] = df
    return dataframes

In [13]:
data_dictionary = pd.read_csv(r"/content/drive/MyDrive/AmEx/details/data_dictionary.csv")

In [20]:
df = pd.read_parquet(r"/content/drive/MyDrive/AmEx/data/train_data.parquet")

In [21]:
test_df = pd.read_parquet(r"/content/drive/MyDrive/AmEx/data/test_data.parquet")

In [22]:
missing_value_count = df.isnull().sum().sort_values( ascending=False)


In [7]:
# features which have more than 75% missing values
features_to_drop = missing_value_count[missing_value_count > 0.75 * len(df)].index.tolist()
print("Features with more than 75% missing values:", features_to_drop)


Features with more than 75% missing values: ['f136', 'f135', 'f112', 'f122', 'f80', 'f360', 'f120', 'f34', 'f19', 'f13', 'f21', 'f14', 'f17', 'f20', 'f15', 'f16', 'f18', 'f84', 'f37', 'f189', 'f221', 'f205', 'f154', 'f176', 'f64', 'f88', 'f66', 'f70', 'f92', 'f220', 'f33', 'f79', 'f36', 'f118', 'f114', 'f81', 'f117', 'f4', 'f121', 'f3', 'f119', 'f116', 'f218']


In [25]:
def print_feature_details(features_to_drop, data_dictionary):
    for feature in features_to_drop:
        feature_info = data_dictionary[data_dictionary['masked_column'] == feature]
        if not feature_info.empty:
            for _, row in feature_info.iterrows():
                print(f"ID: {row['masked_column']}\nDescription: {row['Description']}\nType: {row['Type']}\n{'-'*60}")
        else:
            print(f"ID: {feature} - No information available in data dictionary.\n{'-'*60}")

In [9]:
# Rearrange the features in ascending order of missing values in a new DataFrame
sorted_columns = missing_value_count.sort_values(ascending=True).index.tolist()
df_sorted = df[sorted_columns]
df_sorted.sample(10)

,id1,id2,id3,id4,id5,y,f350,f349,f307,f306,...,f13,f19,f34,f120,f360,f80,f122,f112,f135,f136
711884,1890919_224505_16-23_2023-11-03 09:26:59.940,1890919,224505,2023-11-03 09:26:59.940,2023-11-03,0,34018,6,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
310047,1818194_60448_16-23_2023-11-03 21:10:19.466,1818194,60448,2023-11-03 21:10:19.466,2023-11-03,0,76127,6,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
692730,1707942_5420674_16-23_2023-11-01 13:34:19.402,1707942,5420674,2023-11-01 13:34:19.402,2023-11-01,0,48848,4,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
470325,1456514_920103_16-23_2023-11-02 04:05:20.528,1456514,920103,2023-11-02 04:05:20.528,2023-11-02,0,14719,5,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
757411,1209274_5420674_16-23_2023-11-01 09:46:09.536,1209274,5420674,2023-11-01 09:46:09.536,2023-11-01,0,35155,4,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
119388,1739351_56483_16-23_2023-11-01 21:12:14.939,1739351,56483,2023-11-01 21:12:14.939,2023-11-01,0,76295,4,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
133033,1700033_707815_16-23_2023-11-03 06:48:51.293,1700033,707815,2023-11-03 06:48:51.293,2023-11-03,0,24510,6,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
661222,1248085_75531_16-23_2023-11-01 08:01:48.023,1248085,75531,2023-11-01 08:01:48.023,2023-11-01,0,28902,4,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
343699,1052281_5718_16-23_2023-11-03 12:43:07.319,1052281,5718,2023-11-03 12:43:07.319,2023-11-03,0,45778,6,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None
757108,1106790_79016_16-23_2023-11-02 11:43:39.748,1106790,79016,2023-11-02 11:43:39.748,2023-11-02,0,42217,5,0.0,0.0,...,None,None,None,None,None,None,None,None,None,None


In [12]:
data_dictionary = pd.read_csv(r"/content/drive/MyDrive/AmEx/details/data_dictionary.csv")

In [10]:
def impute_missing_values(df, strategy='median'):
    df_imputed = df.copy()
    for column in df_imputed.columns:
        if df_imputed[column].isnull().any():
            if df_imputed[column].dtype == 'float64':
                if strategy == 'mean':
                    df_imputed[column].fillna(df_imputed[column].mean(), inplace=True)
                elif strategy == 'median':
                    df_imputed[column].fillna(df_imputed[column].median(), inplace=True)
    return df_imputed

In [11]:
# converting the column of df to respective dtypes as mentioned in the data dictionary
def convert_column_types(df, data_dictionary):
    type_map = {
        'Numerical': 'float64',
        'Categorical': 'category',
        'Key': 'category',
        'Label': 'category',
        'One hot encoded': 'category',
        '-': 'object',
    }
    for column in df.columns:
        if column in data_dictionary['masked_column'].values:
            dtype = data_dictionary.loc[data_dictionary['masked_column'] == column, 'Type'].iloc[0]
            if dtype in type_map:
                try:
                    df[column] = df[column].astype(type_map[dtype])
                except Exception as e:
                    print(f"Could not convert {column} to {type_map[dtype]}: {e}")
    return df

In [12]:
def print_object_columns(df):
    dtype_groups = {}
    for col in df.columns:
        dtype_str = str(df[col].dtype)
        dtype_groups.setdefault(dtype_str, []).append(col)
    for dtype, cols in dtype_groups.items():
        print(f"\nColumns with dtype '{dtype}' ({len(cols)} columns):")
        for col in cols:
            print(f"  - {col}")

In [51]:
# rearranging in ascending order of missing values less than 25%
sorted_columns = missing_value_count[missing_value_count < 0.40 * len(df)].sort_values(ascending=True).index.tolist()
df_25 = df[sorted_columns]

In [52]:
df_25 = convert_column_types(df_25, data_dictionary)

/tmp/ipython-input-11-4006871157.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype(type_map[dtype])
/tmp/ipython-input-11-4006871157.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype(type_map[dtype])
/tmp/ipython-input-11-4006871157.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

Could not convert id4 to float64: could not convert string to float: '2023-11-02 22:22:00.042'
Could not convert id5 to float64: could not convert string to float: '2023-11-02'


/tmp/ipython-input-11-4006871157.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype(type_map[dtype])
/tmp/ipython-input-11-4006871157.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype(type_map[dtype])
/tmp/ipython-input-11-4006871157.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [53]:
df_25.dtypes.value_counts()

,count
float64,199
category,48
category,36
object,3
category,2
category,1
category,1
category,1
category,1
category,1


In [17]:
print_object_columns(df_25)


Columns with dtype 'category' (91 columns):
  - f349
  - id1
  - id3
  - y
  - f226
  - f227
  - f229
  - f230
  - f266
  - f265
  - f264
  - f263
  - f262
  - f261
  - f260
  - f259
  - f282
  - f281
  - f280
  - f279
  - f278
  - f277
  - f276
  - f275
  - f267
  - f268
  - f269
  - f270
  - f271
  - f304
  - f307
  - f306
  - f294
  - f293
  - f285
  - f297
  - f298
  - f299
  - f308
  - f309
  - f295
  - f296
  - f288
  - f287
  - f286
  - f292
  - f289
  - f290
  - f274
  - f273
  - f272
  - f291
  - f250
  - f305
  - f302
  - f303
  - f239
  - f240
  - f241
  - f242
  - f243
  - f244
  - f245
  - f246
  - f231
  - f232
  - f233
  - f234
  - f235
  - f228
  - f237
  - f238
  - f247
  - f248
  - f249
  - f284
  - f283
  - f258
  - f257
  - f256
  - f255
  - f254
  - f252
  - f236
  - f251
  - f300
  - f253
  - f301
  - f354
  - f52
  - f50

Columns with dtype 'float64' (160 columns):
  - f350
  - f335
  - f223
  - f225
  - f224
  - f333
  - f332
  - f334
  - f201
  - f203
  - f204

In [54]:
missing_cols = df_25.columns[df_25.isnull().any()]
dtype_counts = df_25[missing_cols].dtypes.value_counts()
print("Number of columns with missing values grouped by dtype in df_25:")
print(dtype_counts)

Number of columns with missing values grouped by dtype in df_25:
float64     198
category     48
category     36
category      2
category      1
category      1
category      1
category      1
category      1
Name: count, dtype: int64


In [55]:
df_25_imputed = impute_missing_values(df_25, strategy='median')

/tmp/ipython-input-10-1598317052.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_imputed[column].fillna(df_imputed[column].median(), inplace=True)
/tmp/ipython-input-10-1598317052.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

In [56]:
missing_cols_imputed = df_25_imputed.columns[df_25_imputed.isnull().any()]
dtype_counts_imputed = df_25_imputed[missing_cols_imputed].dtypes.value_counts()
print("Number of columns with missing values grouped by dtype in df_25:")
print(dtype_counts_imputed)

Number of columns with missing values grouped by dtype in df_25:
category    48
category    36
category     2
category     1
category     1
category     1
category     1
category     1
Name: count, dtype: int64


In [59]:

df_25_imputed.isnull().sum().sort_values(ascending=False)[:90]

,0
f55,254467
f53,254467
f56,254467
f54,254467
f50,190663
...,...
f282,18
f281,18
f248,18
f250,18


In [60]:
def impute_one_hot_encoded_with_median(df, data_dictionary):
    skip_cols = {"f50", "f52", "f354"}
    one_hot_cols = data_dictionary[data_dictionary['Type'].str.lower().str.contains('one hot encoded')]['masked_column'].tolist()

    cols_to_impute = [
        col for col in one_hot_cols
        if col in df.columns and col not in skip_cols and df[col].isnull().any()
    ]

    df_imputed = df.copy()

    for col in cols_to_impute:
        # Coerce to numeric if needed
        df_imputed[col] = pd.to_numeric(df_imputed[col], errors='coerce')
        median_val = df_imputed[col].median()
        if pd.notna(median_val):
            df_imputed[col] = df_imputed[col].fillna(median_val)
        else:
            print(f"⚠️ Skipping column {col} — median is NaN (possibly all missing)")

    return df_imputed


In [61]:
df_imputed_all = impute_one_hot_encoded_with_median(df_25_imputed, data_dictionary)

In [62]:
missing_cols_imputed = df_imputed_all.columns[df_imputed_all.isnull().any()]
dtype_counts_imputed = df_imputed_all[missing_cols_imputed].dtypes.value_counts()
print("Number of columns with missing values grouped by dtype in df_25:")
print(dtype_counts_imputed)

Number of columns with missing values grouped by dtype in df_25:
category    2
category    1
category    1
category    1
category    1
category    1
Name: count, dtype: int64


In [69]:
df_imputed_all['f56'].unique()

[NaN, 'G', 'B', 'S', 'D']
Categories (4, object): ['B', 'D', 'G', 'S']

In [65]:
df_imputed_all.isnull().sum().sort_values(ascending=False)[:10]

,0
f54,254467
f56,254467
f53,254467
f55,254467
f52,190663
f50,190663
f354,141991
f150,0
f30,0
f216,0


In [67]:
print_feature_details(df_imputed_all.columns[df_imputed_all.isnull().any()].sort_values(ascending=False), data_dictionary)

ID: f56
Description: H Code Tier
Type: Categorical
Missing Values: 254467
------------------------------------------------------------
ID: f55
Description: HG Vacation Club
Type: Categorical
Missing Values: 254467
------------------------------------------------------------
ID: f54
Description: Honors enrollees with Stay Activity within the last 12 months = A
Type: Categorical
Missing Values: 254467
------------------------------------------------------------
ID: f53
Description: Value assigned to member 1=lowest  5=highest
Type: Categorical
Missing Values: 254467
------------------------------------------------------------
ID: f52
Description: Active Part y1
Type: Categorical
Missing Values: 190663
------------------------------------------------------------
ID: f50
Description: Account Creation Indicator
Type: Categorical
Missing Values: 190663
------------------------------------------------------------
ID: f354
Description: Depicts days since launch
Type: Categorical
Missing Values

In [27]:
import seaborn as sns

In [28]:
df_imputed_all = df_imputed_all.select_dtypes(include=['float64', 'int64'])

In [29]:
df_imputed_all.isnull().sum().sort_values(ascending=False)[:90]

,0
f350,0
f335,0
f223,0
f225,0
f226,0
...,...
f254,0
f252,0
f236,0
f251,0


In [30]:
# correlation plot cluster heatmap function
# def plot_correlation_heatmap(df, title='Correlation Heatmap'):
#     corr = df.corr()
#     plt.figure(figsize=(10, 8))
#     sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
#     plt.title(title)
#     plt.show()

# # correlation heatmap for the DataFrame with imputed values (df_imputed_all)
# if df_imputed_all.empty:
#     print("DataFrame is empty after imputation.")
# else:
#     print("Correlation Heatmap for DataFrame with Imputed Values:")
#     plot_correlation_heatmap(df_imputed_all, title='Correlation Heatmap of Imputed DataFrame')

# plot_correlation_heatmap(df_imputed_all, title='Correlation Heatmap of Imputed DataFrame')


In [14]:
event_df = pd.read_parquet("/content/drive/MyDrive/AmEx/data/add_event.parquet")

In [15]:
trans_df = pd.read_parquet("/content/drive/MyDrive/AmEx/data/add_trans.parquet")

In [16]:
offer_df = pd.read_parquet("/content/drive/MyDrive/AmEx/data/offer_metadata.parquet")

In [17]:
event_df.sample(10)

,id2,id3,id6,id4,id7
11395859,2824706,699005,OffersTab,2023-10-29 19:58:29.931,None
4800408,2751185,87731,Tiles,2023-10-28 20:58:36.451,None
16121715,2605204,1398,Tiles,2023-11-02 06:09:19.338,None
13462489,2100473,542986322,Tiles,2023-10-26 05:40:47.331,None
3439162,2201965,34826,Tiles,2023-10-29 16:03:20.123,None
10700633,2261067,87102,OffersTab,2023-11-01 18:22:23.079,None
11744620,2591352,76604,Tiles,2023-11-01 17:59:42.463,None
18263490,2281435,62259,Tiles,2023-10-26 04:39:05.716,None
7752227,2651028,958700,Tiles,2023-11-01 10:54:47.438,None
17592846,2531650,77410,OffersTab,2023-10-29 10:06:05.462,None


In [26]:
for col in event_df.columns:
    print(f"{col} : {event_df[col].nunique()}")
    print_feature_details([col], data_dictionary)

id2 : 428195
ID: id2
Description: Customer ID (masked)
Type: -
------------------------------------------------------------
id3 : 923
ID: id3
Description: Offer ID
Type: Categorical
------------------------------------------------------------
id6 : 3
ID: id6 - No information available in data dictionary.
------------------------------------------------------------
id4 : 20794419
ID: id4
Description: Event timestamp
Type: Numerical
------------------------------------------------------------
id7 : 398756
ID: id7 - No information available in data dictionary.
------------------------------------------------------------


In [27]:
# prompt: write a function which tell me all the necessary things about the dataset and which i should know

def explore_dataset(df:pd.DataFrame, df_name="DataFrame"):
    """
    Prints key information about a Pandas DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to explore.
        df_name (str): The name of the DataFrame (for printing purposes).
    """
    print(f"--- Exploring {df_name} ---")

    # Basic Info
    print(f"\nShape: {df.shape}")
    print("\nInfo:")
    df.info()

    # Descriptive Statistics
    print("\nDescriptive Statistics:")
    print(df.describe(include='all'))

    # Missing Values
    print("\nMissing Values (Count and Percentage):")
    missing_count = df.isnull().sum()
    missing_percent = (missing_count / len(df)) * 100
    missing_info = pd.DataFrame({
        'Missing Count': missing_count,
        'Missing Percentage (%)': missing_percent
    }).sort_values(by='Missing Count', ascending=False)
    print(missing_info[missing_info['Missing Count'] > 0])

    # Data Types
    print("\nData Types:")
    print(df.dtypes.value_counts())

    # Unique Values (for non-numeric columns or columns with few unique values)
    print("\nUnique Values (for select columns):")
    for col in df.columns:
        if df[col].dtype in ['object', 'category'] or df[col].nunique() < 20:
            print(f"- {col}: {df[col].nunique()} unique values")
            feature_info = data_dictionary[data_dictionary['masked_column'] == col]
            if not feature_info.empty:
                for _, row in feature_info.iterrows():
                    print(f"- Description: {row['Description']}\n- Type: {row['Type']}")
            else:
                print(f"ID: {col} - No information available in data dictionary.\n{'-'*60}")
            if df[col].nunique() <= 10: # Print value counts for columns with very few unique values
                print(df[col].value_counts(dropna=False))

    # Check for duplicate rows
    print(f"\nNumber of duplicate rows: {df.duplicated().sum()}")

    print(f"\n--- End of Exploration for {df_name} ---")



In [49]:
# Example usage with your dataframes:
# explore_dataset(df, "Train Data")
# explore_dataset(test_df, "Test Data")
# explore_dataset(event_df, "Add Event Data")
# explore_dataset(trans_df, "Add Trans Data")
# explore_dataset(offer_df, "Offer Metadata")
# explore_dataset(data_dictionary, "Data Dictionary")
# explore_dataset(df_25, "df_25 (Imputed up to 25% missing)")
# explore_dataset(df_imputed_all, "df_imputed_all (Final Imputed)")

In [29]:
print_feature_details(offer_df.columns, data_dictionary)

ID: id3
Description: Offer ID
Type: Categorical
------------------------------------------------------------
ID: id9 - No information available in data dictionary.
------------------------------------------------------------
ID: f375 - No information available in data dictionary.
------------------------------------------------------------
ID: f376 - No information available in data dictionary.
------------------------------------------------------------
ID: f377 - No information available in data dictionary.
------------------------------------------------------------
ID: id10 - No information available in data dictionary.
------------------------------------------------------------
ID: id11 - No information available in data dictionary.
------------------------------------------------------------
ID: f378 - No information available in data dictionary.
------------------------------------------------------------
ID: f374 - No information available in data dictionary.
-----------------

In [72]:
offer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id3     4164 non-null   int32  
 1   id9     4164 non-null   object 
 2   f375    4164 non-null   int32  
 3   f376    978 non-null    float64
 4   f377    0 non-null      object 
 5   id10    4164 non-null   object 
 6   id11    0 non-null      object 
 7   f378    4164 non-null   object 
 8   f374    3887 non-null   object 
 9   id8     3887 non-null   object 
 10  id12    4164 non-null   object 
 11  id13    4164 non-null   object 
dtypes: float64(1), int32(2), object(9)
memory usage: 358.0+ KB


In [70]:
event_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21457473 entries, 0 to 21457472
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id2     int32 
 1   id3     object
 2   id6     object
 3   id4     object
 4   id7     object
dtypes: int32(1), object(4)
memory usage: 736.7+ MB


In [71]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6339465 entries, 0 to 6339464
Data columns (total 9 columns):
 #   Column  Dtype  
---  ------  -----  
 0   id2     int32  
 1   f367    float64
 2   f368    object 
 3   f369    object 
 4   f370    object 
 5   f371    object 
 6   f372    object 
 7   id8     object 
 8   f374    object 
dtypes: float64(1), int32(1), object(7)
memory usage: 411.1+ MB


In [50]:
explore_dataset(df_25, "df_25 (Imputed up to 25% missing)")

--- Exploring df_25 (Imputed up to 25% missing) ---

Shape: (770164, 254)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770164 entries, 0 to 770163
Columns: 254 entries, f349 to f313
dtypes: category(91), float64(160), object(3)
memory usage: 1.0+ GB

Descriptive Statistics:
          f349           f350                                          id1  \
count   770164  770164.000000                                       770164   
unique       4            NaN                                       770164   
top          4            NaN  1910568_73186_16-23_2023-11-03 23:10:57.764   
freq    301584            NaN                                            1   
mean       NaN   43888.750377                                          NaN   
std        NaN   19134.165795                                          NaN   
min        NaN      23.000000                                          NaN   
25%        NaN   28974.000000                                          NaN   
50%        

In [42]:
df_25['f224'].nunique()

180